# **Predicting selling price of the vehicle**

**Dataset Description:
The "Vehicle Sales and Market Trends Dataset" provides a comprehensive collection of information pertaining to the sales transactions of various vehicles. This dataset encompasses details such as the year, make, model, trim, body type, transmission type, VIN (Vehicle Identification Number), state of registration, condition rating, odometer reading, exterior and interior colors, seller information, Manheim Market Report (MMR) values, selling prices, and sale dates.**

In [ ]:
from IPython.display import Image, display

# Display the image
image_path = '/kaggle/input/image-data-dep/cap_data.jpg'  
display(Image(filename=image_path))

image_path = '/kaggle/input/data-dep-image/cap.jpg' 
display(Image(filename=image_path))

# Importing Essential Libraries & Dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/vehicle-sales-data/car_prices.csv')

In [ ]:
df.head(1)

In [ ]:
df.info()

# Exploratory Data Analysis

In [ ]:
df.select_dtypes(include=np.number).columns

In [ ]:
df.select_dtypes(exclude=np.number).columns

### 5 Point Summary of Numerical Variables

In [ ]:
df.describe().T

### Summary of Categorical Variables

In [ ]:
df.describe(exclude=np.number).T

# Univariate Analysis

## Categorical Variables

In [ ]:
plt.figure(figsize=(15,5));
sns.countplot(x=df.body);
plt.xticks(rotation=90);
plt.title('Count of each body type');

### There seems to be an issue of case sensitive characters in the data which we will clean later.
### Sedan and SUV seem to be most popular body types.

### The following count plots have been capped at top 40 values each because they contain way too many values to be effectively visualized

In [ ]:
j=1
plt.figure(figsize=(10,20))
for i in ['make', 'model', 'trim', 'seller']:
    plt.subplot(4,1,j)
    sns.barplot(x = df[i].value_counts().index[0:40], y=df[i].value_counts()[0:40]);
    plt.xticks(rotation=90);
    plt.title(f'Count of {i}')
    plt.tight_layout();
    j+=1

In [ ]:
df.seller.value_counts()

#### There are 14263 different sellers in this data

In [ ]:
plt.figure(figsize=(10,5));
sns.countplot(x=df.color);
plt.xticks(rotation=90);

#### There are some garbage characters in the column 'color'
#### The countplot of the above column color shows us that the top 5 popular colors when it comes to cars are as follows
#### 1 - Black
#### 2 - White
#### 3 - Silver
#### 4 - Gray
#### 5 - Red

In [ ]:
plt.figure(figsize=(10,5));
sns.countplot(x=df.state);
plt.xticks(rotation=90);

### It can be observed that there are a lot of garbage characters in the column 'state'

In [ ]:
print('Top 5 state according to the figure and value counts is as below')
df.state.value_counts(ascending=False).head()

In [ ]:
plt.figure(figsize=(10,4));
sns.countplot(y=df['interior']);
plt.xticks(rotation=90);

### We can identify from the above plot that there is a garbage value in the column 'interior' which have to be treated.

In [ ]:
plt.figure(figsize=(10,4));
sns.countplot(y=df.transmission);

### Within the column transmission, the value 'sedan' belongs to the column 'body' so we will have to further check the column for data discrepency

## Numerical Variables

In [ ]:
j=1
plt.figure(figsize=(10,10))
for i in ['year', 'condition', 'odometer', 'mmr', 'sellingprice']:
    plt.subplot(3,2,j)
    sns.distplot(df[i]);
    plt.tight_layout()
    plt.xticks(rotation=90);
    plt.title(np.round(df[i].skew(),2))
    j+=1

### Year and Condition have missing values at unusual frequencies. We will further examine these two columns.
### Odometer, MMR & Selling Price are right skewed and there seems to be no discrepency in distribution of these columns.

In [ ]:
np.array(sorted(df.year.unique()))

### By looking at visualization, it looked like there were no values for certain years but by checking the data for unique year values, we can notice that there is some data available for every year, its just that the data for some particular years is very less

In [ ]:
df.condition.value_counts().sort_index()

### The values in 'condition' column seem to be in two ranges i.e. 1-5 and 10-50
### We will be treating the same in bivariate analysis

# Bivariate Analysis

In [ ]:
sns.boxplot(x=df.year, y=df.sellingprice, whis=3.0);
plt.xticks(rotation=90);

#### The selling price of older cars seems to be less which is a very obvious and ordinary situation in used car market

In [ ]:
sns.scatterplot(x=df.condition, y=df.sellingprice);

#### There appears to be a positive relation between condition and selling price. This means that higher condition score has higher selling price.  There are also a few outliers which have have lower selling price despite high condition rating.
#### Some values within selling price are very close to 0, we will further examine this column when doing data cleaning.
#### We can also verify the fact there are two scales of data within a single column, which shall get fixed by changing the scale of either range.

In [ ]:
df['condition_new'] = np.where(df.condition<10, df.condition*10, df.condition)
sns.scatterplot(x=df.condition_new, y=df.sellingprice);

#### The change in scale for condition range 1-5 seems to have worked perfectly
#### There are no suspiciously missing values between dense data points after the treatment

In [ ]:
sns.scatterplot(x=df.odometer, y=df.sellingprice);

#### There appears to be a negative relation between odometer reading and selling price. This means that as the odometer reading (mileage) increases, the selling price tends to decrease.

In [ ]:
sns.scatterplot(x=df.condition_new, y=df.odometer);

In [ ]:
sns.boxplot(x=df.transmission, y=df.sellingprice, showmeans=True);

In [ ]:
df.groupby(by='transmission').sellingprice.mean()

### The graph shows that cars with automatic transmissions have a higher average selling price than cars with manual transmission. The average selling price for a car with an automatic transmission is around 13,540, while the average selling price for a car with a manual transmission is around $11,211.

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(x=df.make, y=df.sellingprice)
plt.xticks(rotation=90);

#### Variations in price across makes: There is a significant variation in average selling price across different car makes. Some makes, like Lexus and Mercedes-Benz, tend to have higher average selling prices than others, like Kia and Hyundai. This likely reflects brand reputation, standard features, and performance differences.

#### Variations within makes: There are also variations in average selling price within each car make. For example, within the Lexus brand, the LS likely has a higher average selling price than the RX. This is likely due to differences in original price, model size, features, and performance.

# Data Cleaning

In [ ]:
df.isnull().sum()

plt.figure(figsize=(20,30))
df['model'].value_counts().sort_values().plot(kind='barh')

## Finding case sensitive duplicates in the data

#### Finding & Fixing lower case duplicates in the feature 'Make'.

In [ ]:
df['make'].nunique(), df['make'].str.lower().nunique()

In [ ]:
df['make']=df['make'].str.lower()

#### Finding & Fixing lower case duplicates in the feature 'Model'.

In [ ]:
df['model'].nunique(), df['model'].str.lower().nunique()

In [ ]:
df['model']=df['model'].str.lower()

#### Finding & Fixing lower case duplicates in the feature 'Trim'.

In [ ]:
df['trim'].nunique(), df['trim'].str.lower().nunique()

In [ ]:
df['trim']=df['trim'].str.lower()

#### Finding & Fixing lower case duplicates in the feature 'Body'.

In [ ]:
df['body'].nunique(), df['body'].str.lower().nunique()

In [ ]:
df['body']=df['body'].str.lower()

## Garbage Values Treatment

In [ ]:
plt.figure(figsize=(6,2))
df['transmission'].value_counts().sort_values().plot(kind='barh');

In [ ]:
df.loc[(df['transmission']=='sedan') | (df['transmission']=='Sedan')]

In [ ]:
df.loc[(df['transmission']=='sedan') | (df['transmission']=='Sedan')].index

###### Indexes of garbage characters.

In [ ]:
[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710]

#### Garbage in feature 'Trim'.

In [ ]:
df.loc[df.loc[(df['transmission']=='sedan') | (df['transmission']=='Sedan')].index,'trim']=np.nan

#### Garbage in feature 'Body'.

In [ ]:
df.loc[df.loc[(df['transmission']=='sedan') | (df['transmission']=='Sedan')].index,'body']=df.loc[df.loc[(df['transmission']=='sedan') | (df['transmission']=='Sedan')].index,'transmission']

#### Garbage in feature 'Transmission'.

In [ ]:
df.loc[df.loc[(df['transmission']=='sedan') | (df['transmission']=='Sedan')].index,'transmission']=df.loc[df.loc[(df['transmission']=='sedan') | (df['transmission']=='Sedan')].index,'vin']

#### Garbage in feature 'Vin'.

In [ ]:
df.loc[df['vin']=='automatic','vin']=df.loc[df['vin']=='automatic','state']

In [ ]:
df.loc[df['vin']=='automatic']

In [ ]:
df.loc[df['vin'].isnull(),'vin']=df.loc[df['vin'].isnull(),'state']

In [ ]:
df.vin.isnull().sum()

In [ ]:
df.loc[[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710]]

#### Garbage in feature 'State'.

In [ ]:
df.loc[[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710],'state']=np.nan

In [ ]:
df.loc[[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710]]

#### Garbage in feature 'Condition'.

In [ ]:
df.loc[[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710],'condition']=df.loc[[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710],'odometer']

#### Garbage in feature 'Odometer'.

In [ ]:
df.loc[[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710],'odometer']=df.loc[[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710],'color']

In [ ]:
df.loc[[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710]]

#### Garbage in feature 'Color'.

In [ ]:
df.loc[[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710],'color']=df.loc[[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710],'interior']

In [ ]:
df.loc[df['color']=='—','color']=np.nan

#### Garbage in feature 'Interior'.

In [ ]:
df.loc[[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710],'interior']=df.loc[[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710],'seller']

In [ ]:
df.loc[df['interior']=='—','interior']=np.nan

#### Garbage in feature 'Seller'.

In [ ]:
df.loc[[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710],'seller']=np.nan
       

#### Garbage in feature 'MMR'.

In [ ]:
df.loc[[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710],'mmr']=df.loc[[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710],'saledate']

#### Garbage in feature 'Saledate'.

In [ ]:
df.loc[[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710],'saledate']=np.nan
       

In [ ]:
df.loc[[408161, 417835, 421289, 424161, 427040, 427043, 434424, 444501, 453794,
       461597, 461612, 492484, 497008, 497011, 499083, 501455, 505299, 505306,
       520461, 528996, 529009, 529013, 529622, 548784, 551222, 554710]]

In [ ]:
#df['sd_new']=pd.to_datetime(df['saledate'],format="%Y-%m-%d %H:%M:%S%z",errors='coerce',utc=True)

In [ ]:
df.isnull().sum()

### Null Value Treatment.

#### Feature : 'Make'.

In [ ]:
df['vin_new']=df['vin'].str[0:3]

In [ ]:
df['vin'].isnull().sum()

In [ ]:
make=dict(df.groupby('vin_new')['make'].describe().top)

In [ ]:
df['vin_new'].isnull().sum()

In [ ]:
df['make_n']=np.where(df['make'].isnull()==True,df['vin_new'].map(make),df['make'])

In [ ]:
df['make_n'].isnull().sum()

In [ ]:
df.loc[df['vin_new']=='1l1','make_n']='Lincoln'

In [ ]:
df.loc[(df['vin_new']=='3d2') | (df['vin_new']=='3d6'),'make_n']='Chrysler'

In [ ]:
df.loc[df['vin_new']=='wdy','make_n']='Daimler Chrysler'

In [ ]:
df.loc[(df['vin_new']=='4gl') | (df['vin_new']=='1ge'),'make_n']='General Motors'

In [ ]:
df.loc[(df['vin_new']=='1jc') | (df['vin_new']=='2j4') ,'make_n']='Jeep'

In [ ]:
df.loc[df['vin_new']=='wda','make_n']='Mercedes Benz'

In [ ]:
df.loc[df['vin_new']=='2fd','make_n']='Ford'

In [ ]:
df['make_n'].isnull().sum()

In [ ]:
df.loc[df['vin'].isnull()]

In [ ]:
df['body']=df['body'].str.lower()

In [ ]:
df.isnull().sum()

#### Feature : 'Model'

In [ ]:
df['vin_m']=df['vin'].str[3:8]

In [ ]:
d_m=dict(df.groupby('vin_m')['model'].describe().top)

In [ ]:
df['model_n']=np.where(df['model'].isnull()==True,df['vin_m'].map(d_m),df['model'])

In [ ]:
df['model_n'].isnull().sum()

In [ ]:
df.loc[df['model_n'].isnull()]

#### Feature : 'Transmission'.

In [ ]:
dict_transmission = dict(df.groupby('vin_m')['transmission'].describe().top)

In [ ]:
df['trans_n']=np.where(df['transmission'].isnull()==True,df['vin_m'].map(dict_transmission),df['transmission'])

In [ ]:
df.groupby('vin_m')['transmission'].describe()

In [ ]:
df['trans_n'].isnull().sum()

In [ ]:
df['trans_n'].value_counts()

In [ ]:
df['transmission'].value_counts()

#### Feature: 'Trim'

In [ ]:
dict_trim = dict(df.groupby('vin_m')['trim'].describe().top)

In [ ]:
df['trim_n']=np.where(df['trim'].isnull()==True,df['vin_m'].map(dict_trim),df['trim'])

In [ ]:
df['trim_n'].isnull().sum()

#### Feature : 'Body'

In [ ]:
dict_body = dict(df.groupby('vin_m')['body'].describe().top)

In [ ]:
df['body_n']=np.where(df['body'].isnull()==True,df['vin_m'].map(dict_body),df['body'])

In [ ]:
df['body_n'].isnull().sum()

In [ ]:
df.groupby('condition')['sellingprice'].describe()

In [ ]:
df['mmr']=df['mmr'].astype(float)

In [ ]:
df.groupby('condition')['mmr'].describe()

In [ ]:
df['condition'].isnull().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df['mmr'].isnull().sum()

In [ ]:
df['saledate'].isnull().sum()

In [ ]:
df.isnull().sum()

#### Feature : 'Color'.

In [ ]:
df.loc[(df['color'].isnull()) & (df['interior'].isnull())]

In [ ]:
df.groupby('color')['interior'].describe().head(50)

In [ ]:
df.groupby(by=['make_n','model_n','trans_n'])['color'].describe().head(50)

In [ ]:
df.groupby(by=['make_n','model_n','body_n','trans_n'])['color'].describe().head(50)

In [ ]:
df.groupby(by=['make_n','model_n','trans_n','interior'])['color'].describe()

In [ ]:
df.groupby(by=['body_n'])['color'].describe().head(50)

In [ ]:
df.groupby(by=['make_n','model_n','trans_n','interior'])['color'].describe()

In [ ]:
df['color'].isnull().sum()

In [ ]:
df.groupby('model_n')['color'].describe().head(50)

In [ ]:
d_c=dict(df.groupby('vin_m')['color'].describe().top)

In [ ]:
df['color_n']=np.where(df['color'].isnull()==True,df['vin_m'].map(d_c),df['color'])

In [ ]:
df['color'].value_counts()

In [ ]:
df['color_n'].value_counts()

In [ ]:
df.isnull().sum()

#### Feature : 'Interior'

In [ ]:
df['interior'].isnull().sum()

In [ ]:
df.groupby('vin_m')['interior'].describe()

In [ ]:
d_i=dict(df.groupby('vin_m')['interior'].describe().top)

In [ ]:
df['interior_n']=np.where(df['interior'].isnull()==True,df['vin_m'].map(d_i),df['interior'])

In [ ]:
df['interior_n'].isnull().sum()

In [ ]:
df.isnull().sum()

#### Feature : 'Condition'

In [ ]:
df['condition'] = np.where(df.condition<10, df.condition*10, df.condition)

#### Feature : 'odometer'

In [ ]:
df['odometer']=df['odometer'].astype(float)

In [ ]:
df.loc[(df['odometer'] < 50) & (df['condition'].isnull())]

In [ ]:
df.loc[(df['odometer'] < 50) & (df['condition'].isnull()),'condition']=df.loc[(df['odometer'] < 50) & (df['condition'].isnull()),'odometer']

In [ ]:
df.loc[(df['odometer'] < 50),'odometer' ]=np.nan

In [ ]:
df.isnull().sum()

In [ ]:
d_c=dict(df.groupby('year')['condition'].median())

In [ ]:
df['cond_n']=np.where(df['condition'].isnull()==True,df['year'].map(d_c),df['condition'])

In [ ]:
df['cond_n'].isnull().sum()

#### Feature : 'Body_n'.

In [ ]:
df.isnull().sum()

In [ ]:
d2=dict(df.groupby('model_n')['body_n'].describe().top)

In [ ]:
df['body_n']=np.where(df['body_n'].isnull()==True,df['model_n'].map(d2),df['body_n'])

In [ ]:
df['body_n'].isnull().sum()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.isnull().sum()

### New DataFrame after Data Cleaning

In [ ]:
df_new=df[['year','make_n','model_n','trim_n','body_n','trans_n','vin','state','cond_n','odometer','color_n','interior_n','seller','mmr','sellingprice','saledate']]

In [ ]:
df_new.isnull().sum()

In [ ]:
df_new.shape

### Dropping rows with null values

In [ ]:
df_new=df_new.dropna(ignore_index=True)

In [ ]:
df_new.shape

In [ ]:
df_new.isnull().sum()

In [ ]:
df_new

### Duplicates in Make Column

In [ ]:
df['make_n']=df['make_n'].str.lower()

In [ ]:
df['make_n'].nunique()

In [ ]:
df['make_n'].unique()

In [ ]:
df_new['make_n']=df_new['make_n'].replace(['mercedes-benz','mercedes-b','mercedes benz'],'mercedes')

In [ ]:
df_new['make_n']=df_new['make_n'].replace('chev truck','chevrolet')

In [ ]:
df_new['make_n']=df_new['make_n'].replace('gmc truck','gmc')

In [ ]:
df_new['make_n']=df_new['make_n'].replace('vw','volkswagen')

In [ ]:
df_new['make_n']=df_new['make_n'].replace(['ford tk','ford truck'],'ford')

In [ ]:
df_new['make_n'].nunique()

# Data Frame after Data Cleaning

In [ ]:
df_new

# Feature Engineering

### New Feature : Country of Manufacture

In [ ]:
dict_vin = {
    '1': 'USA', 
    '2': 'Canada', 
    '3': 'Mexico', 
    '4': 'USA', 
    '5': 'USA',
    '6': 'Australia',
    '7': 'New Zealand',
    '8': 'Argentina / Chile / Ecuador / Venezuela / Bolivia', 
    '9': 'Brazil / Colombia / Uruguay',
    'a': 'South Africa', 
    'b': 'Uganda', 
    'j': 'Japan', 
    'k': 'South Korea / Israel',
    'n': 'Turkey', 
    'm': 'India / Indonesia / Thailand / Myanmar / Kazakhstan',
    's': 'England',
    't': 'Switzerland / Czech Republic / Hungary / Portugal',
    'v': 'Austria / France / Spain / Serbia / Croatia',
    'w': 'Germany', 
    'y': 'Sweden/Finland',
    'z': 'Italy / Slovenia / Russia'
}

In [ ]:
df_new['country_manu']=df_new['vin'].str[0:1].map(dict_vin)

### New Feature : Years Driven

In [ ]:
df_new['year2']=pd.to_datetime(df_new['saledate'],format="mixed",utc=True).dt.year

In [ ]:
df_new['n_years']=df_new['year2']-df_new['year']

In [ ]:
df_new=df_new.drop('year2',axis=1)

In [ ]:
df_new['n_years']

In [ ]:
df_new['n_years']=np.where(df_new['n_years']==-1,0,df_new['n_years'])

In [ ]:
df_new['n_years'].value_counts().sort_index()

In [ ]:
df_new.shape

In [ ]:
df_new.isnull().sum()

# Pickle File 1

In [ ]:
import pickle

# Define multiple lists
make = df_new.make_n.unique()
model = df_new.model_n.unique()
trim = df_new.trim_n.unique()
body = df_new.body_n.unique()
transmission = df_new.trans_n.unique()
state = df_new.state.unique()
color = df_new.color_n.unique()
interior = df_new.interior_n.unique()

# Save multiple lists to a single pickle file
unique_col_values = (make, model, trim, body, transmission, state,
                     color, interior)
with open('multiple_lists.pkl', 'wb') as f:
    pickle.dump(unique_col_values, f)

# Statistical Tests

In [ ]:
from scipy.stats import shapiro,levene,kruskal,mannwhitneyu,spearmanr
import warnings
warnings.filterwarnings('ignore')

**Two assumptions of parametric tests:**

**Data is normally distributed and the variances of the data are approximately equal across groups or populations being compared.**

In [ ]:
# Ho: Data is normal
#Ha: Data is not normal

In [ ]:
p=shapiro(df_new['sellingprice'])[1]
if p>0.05:
    print('Ho: Data is Normal')
else:
    print('Ha: Data is not Normal')
p

The first assumption is violated so we do not go for second one.

**Since the Data is not normal so in this case we have to perform non-perametric tests for hypothesis testing.**

In [ ]:
# H0: both the columns are not related (independent)
# Ha: Both the columns are related (dependent)
rho_odometer, pval_odometer = spearmanr(df_new['odometer'], df_new['sellingprice'])
print(f"Spearman's correlation between odometer and sellingprice: {rho_odometer}")
print(f"P-value: {pval_odometer}")

# Calculate Spearman's correlation coefficient and p-value for 'mmr' and 'sellingprice'
rho_mmr, pval_mmr = spearmanr(df_new['mmr'], df_new['sellingprice'])
print(f"Spearman's correlation between mmr and sellingprice: {rho_mmr}")
print(f"P-value: {pval_mmr}")

**In Both case we reject the null Hypothesis**

**Means columns are significant**

In [ ]:
samp1 = df_new[df_new['trans_n']=='automatic']['sellingprice']
samp2 = df_new[df_new['trans_n']=='manual']['sellingprice']

p = mannwhitneyu(samp1,samp2)[1]
if p > 0.05:
    print('Null Hypothesis (Ho): The mean selling prices of cars with automatic and manual transmissions are equal.')
else:
    print('Alternative Hypothesis (Ha): The mean selling prices of cars with automatic and manual transmissions are different.')
print(f'p-value: {p}')

In [ ]:
for j in ['year','make_n','model_n','trim_n','body_n','state','cond_n','color_n','interior_n','country_manu','n_years']:
    grouped_data = [df_new.loc[df_new[j] == i, 'sellingprice']  for i in df_new[j].unique()]
    p = kruskal(*grouped_data)[1]

    if p>0.05:
        print('Ho: There is no relation between sellingprice and',j)
    else:
        print('Ha:There is significant relation between sellingprice and',j)
    print(f'p_value for {j} is {p}')

# Encoding

### (1) Body

In [ ]:
dict_body = df_new['body_n'].value_counts(normalize=True)

In [ ]:
df_new['body_n']=df_new['body_n'].map(dict_body)

### (2) Transmission

In [ ]:
dict_transmission = {'automatic':1, 'manual':0}

In [ ]:
df_new['trans_n']=np.where(df_new['trans_n']=='automatic',1,0)

### (3) Color + Interior (Encoded later)
#### Columns combined due to multicolinearity

In [ ]:
df_new['color+interior']=df_new['color_n']+' '+df_new['interior_n']

In [ ]:
df_new=df_new.drop(['color_n','interior_n'],axis=1)

In [ ]:
df_new['color+interior'].value_counts()

### (4) State

In [ ]:
import category_encoders as ce

In [ ]:
df_new['state'].nunique()

In [ ]:
state_encoder = ce.BinaryEncoder(cols=['state'],return_df=True)

In [ ]:
data_encoded = state_encoder.fit_transform(df_new)

In [ ]:
data_encoded.head(1)

In [ ]:
data_encoded.shape

In [ ]:
from sklearn.model_selection import train_test_split

#### Encoding Features : 'make_n', 'model_n', 'trim_n', 'color+interior','country_manu'

In [ ]:
X=data_encoded.drop(['vin','seller','sellingprice','saledate'],axis=1)
y=data_encoded['sellingprice']

In [ ]:
xtrain, xtest, ytrain, ytest= train_test_split(X,y,train_size=0.70,random_state=10)

In [ ]:
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

In [ ]:
cols=['make_n', 'model_n', 'trim_n', 'color+interior','country_manu']

In [ ]:
encoder=ce.TargetEncoder(cols=['make_n', 'model_n', 'trim_n','color+interior','country_manu'],return_df=True)

In [ ]:
xtrain_encoded=encoder.fit_transform(xtrain[cols],ytrain)

In [ ]:
 xtrain_encoded

In [ ]:
xtrain=xtrain.drop(cols,axis=1)

In [ ]:
xtrain_final=pd.concat([xtrain,xtrain_encoded],axis=1)

In [ ]:
xtrain_final.head()

## Test data

In [ ]:
xtest_encoded=encoder.transform(xtest[cols])

In [ ]:
xtest_encoded

In [ ]:
xtest=xtest.drop(cols,axis=1)

In [ ]:
xtest_final=pd.concat([xtest,xtest_encoded],axis=1)

In [ ]:
xtrain_final.shape, xtest_final.shape, ytrain.shape, ytest.shape

## Encoding completed for Train & Test Data

# Pickle File 2

In [ ]:
import pickle

encoding_data = (dict_body, dict_transmission, state_encoder, encoder)

# Save the tuple to a pickle file
with open('encoding_data.pkl', 'wb') as f:
    pickle.dump(encoding_data, f)

# Scaling and Transformation

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc=StandardScaler()

In [ ]:
cols=['odometer','mmr']

In [ ]:
a=sc.fit_transform(xtrain_final[cols])

In [ ]:
xtrain_final['od_n']=a[:,0]

In [ ]:
xtrain_final['mmr_n']=a[:,1]

In [ ]:
xtrain_final.shape

In [ ]:
xtrain_final.head()

In [ ]:
xtrain_final_sc=xtrain_final.drop(['odometer','mmr'],axis=1)
xtrain_final=xtrain_final.drop(['od_n','mmr_n'],axis=1)

In [ ]:
xtrain_final_sc.shape , xtrain_final.shape

In [ ]:
a1=sc.transform(xtest_final[cols])

In [ ]:
xtest_final['od_n']=a1[:,0]
xtest_final['mmr_n']=a1[:,1]

In [ ]:
xtest_final_sc=xtest_final.drop(['odometer','mmr'],axis=1)
xtest_final=xtest_final.drop(['od_n','mmr_n'],axis=1)

In [ ]:
xtest_final.shape , xtest_final_sc.shape 

### Transformation

In [ ]:
sns.distplot((ytrain))
plt.title(f'Skewness: {np.round(ytrain.skew(),2)}')


In [ ]:
sns.distplot(np.sqrt(ytrain))
plt.title(f'Skewness: {np.round(np.sqrt(ytrain).skew(),2)}')


In [ ]:
ytrain_tra=np.sqrt(ytrain)

In [ ]:
sns.distplot(ytest)
plt.title(f'Skewness: {np.round(ytest.skew(),2)}')

In [ ]:
sns.distplot(np.sqrt(ytest))
plt.title(f'Skewness: {np.round(np.sqrt(ytest).skew(),2)}')

In [ ]:
ytest_tra=np.sqrt(ytest)

## Linear Regression

### Assumptions of Linear Regression

**Before model building**

* Target Variable must be numeric.

In [ ]:
ytrain.describe()

* Absence of Multicollinearity.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
xtrain_final.shape

In [ ]:
vif=[]
for i in range(xtrain_final.shape[1]):
    vif.append(variance_inflation_factor(xtrain_final.values,exog_idx=i))


In [ ]:
pd.DataFrame(vif,xtrain_final.columns)

In [ ]:
xtrain_final=xtrain_final.drop(['country_manu','year'],axis=1)

In [ ]:
vif=[]
for i in range(xtrain_final.shape[1]):
    vif.append(variance_inflation_factor(xtrain_final.values,exog_idx=i))


In [ ]:
pd.DataFrame(vif,xtrain_final.columns)

In [ ]:
vif=[]
for i in range(xtrain_final_sc.shape[1]):
    vif.append(variance_inflation_factor(xtrain_final_sc.values,exog_idx=i))
pd.DataFrame(vif,xtrain_final_sc.columns)

In [ ]:
xtrain_final_sc=xtrain_final_sc.drop(['country_manu','year'],axis=1)

In [ ]:
vif=[]
for i in range(xtrain_final_sc.shape[1]):
    vif.append(variance_inflation_factor(xtrain_final_sc.values,exog_idx=i))
pd.concat([pd.DataFrame(vif,xtrain_final_sc.columns),pd.DataFrame(vif,xtrain_final.columns)],axis=1)

we can clearly see the difference in vif in scaled data and in unscaled data.

In [ ]:
import statsmodels.api as sma

Now we make a base model on scaled data/normaled data and on unscaled data/unnormalised data. 

* **Modelling on unscaled data.**

In [ ]:
xtrain_final_x=sma.add_constant(xtrain_final)
base_model=sma.OLS(ytrain,xtrain_final_x).fit()

In [ ]:
base_model.summary()

* **Modelling on scaled data.**

In [ ]:
xtrain_final_sc_x=sma.add_constant(xtrain_final)
base_model_sc_df=sma.OLS(ytrain_tra,xtrain_final_sc_x).fit()
base_model_sc_df.summary()

As the performance metric r2_score is maximum on unscaled data we will use that one only.

#### Assumptions after the model

**Linear relationship between independent and dependent.**

In [ ]:
plt.figure(figsize=(10,20))
t=1
for i in xtrain_final_sc:
    plt.subplot(9,2,t)
    sns.scatterplot(x=xtrain_final_sc[i],y=base_model_sc_df.resid)
    plt.title(f'{i} vs residuals')
    t+=1
plt.tight_layout()
plt.show()
    

In [ ]:
from statsmodels.stats.diagnostic import linear_rainbow
linear_rainbow(base_model_sc_df)

* Null rejected.
* There is no linear relationship bw independent and dependent variables.

**Absence of Auto-Collinearity**
* Durbin Watson : 1.998
* Assumption of absence of auto collinearity true.

**Homoskedasticity of errors.**

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan
het_breuschpagan(base_model_sc_df.resid,base_model_sc_df.model.exog)

* The statistical test rejects the null hypothesis of hetroskedasticity.
* The error terms are homoskedastic.

In [ ]:
residuals = base_model_sc_df.resid
fitted_values = base_model_sc_df.fittedvalues

plt.scatter(fitted_values, residuals)
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.title("Residuals vs. Fitted Values Plot")
plt.axhline(y=0, color='r', linestyle='-')
plt.show()

* The graphical representation shows presence of heteroskedasticity in the error terms.
* Assumption : Error terms must be normally distributed

In [ ]:
import scipy.stats as stats

In [ ]:
stats.probplot(base_model_sc_df.resid,plot=plt)
plt.show()

* The error terms are not normally distributed.

### Checking performance of the base model on the train set.

# Creating Function for validation: 

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

def validation(test,pred):
    print(f'R-Squared : {r2_score(test,pred)}')
    print(f'Mean Squared Error : {mean_squared_error(test,pred)}')
    print(f'Root Mean Squared Error : {np.sqrt(mean_squared_error(test,pred))}')

In [ ]:
ytrain_pred=base_model.predict(xtrain_final_x)

In [ ]:
validation(ytrain,ytrain_pred)

### Checking performance of the base model on the test set.

In [ ]:
xtest_final=xtest_final.drop(['year','country_manu'],axis=1)

In [ ]:
xtest_final_x=sma.add_constant(xtest_final)
ytest_pred=base_model.predict(xtest_final_x)

In [ ]:
validation(ytest,ytest_pred)

**R2 is still the same showing absence of overfitting.**

* Here, we will do cross validation.

In [ ]:
from sklearn.model_selection import KFold,cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor
from xgboost import XGBRegressor

In [ ]:
lr=LinearRegression()
cv = KFold(n_splits=5,shuffle=True,random_state=1001)
scores = cross_val_score(estimator=lr,X =xtrain_final, y = ytrain, cv=cv, scoring='r2', n_jobs=-1)
scores

In [ ]:
xtrain_final

In [ ]:
# base model
def base_models():
    models = dict()
    models['LR'] = LinearRegression()
    models['DTREE'] = DecisionTreeRegressor()
    models['RF'] = RandomForestRegressor()
    models['GBM'] = GradientBoostingRegressor()
    models['XGB'] = XGBRegressor()
    models['ADA'] = AdaBoostRegressor()
    return models

In [ ]:
from sklearn.model_selection import KFold,cross_val_score

def eval_models(model):
    cv = KFold(n_splits=5,shuffle=True,random_state=1001)
    scores = cross_val_score(estimator=model,X =xtrain_final,y = ytrain,cv=cv,scoring='r2',
                            n_jobs=-1)
    return scores




In [ ]:
# lets use the functions to calculate the scores........

models = base_models()

result , names = list() , list()

for name , mymodel in models.items():
    finalscore = eval_models(mymodel)
    result.append(finalscore)
    names.append(name)
    print(name,np.mean(result))

In [ ]:
dt = DecisionTreeRegressor()
dt.fit(xtrain_final, ytrain)
dtpred = dt.predict(xtest_final)
validation(ytest, dtpred)

In [ ]:
feature_imp = pd.DataFrame()
feature_imp['Feature']=xtrain_final.columns
feature_imp['Importance']=dt.feature_importances_

feature_imp = feature_imp.sort_values('Importance',ascending=False)
sns.barplot(x='Importance', y='Feature', data=feature_imp)
plt.show()

## Ensemble Modelling

In [ ]:
rf = RandomForestRegressor(n_jobs=-1)
rf.fit(xtrain_final, ytrain)
rfpred = rf.predict(xtest_final)
validation(ytest, rfpred)

In [ ]:
feature_imp = pd.DataFrame()
feature_imp['Feature']=xtrain_final.columns
feature_imp['Importance']=rf.feature_importances_

feature_imp = feature_imp.sort_values('Importance',ascending=False)
sns.barplot(x='Importance', y='Feature', data=feature_imp)
plt.show()

In [ ]:
gbm = GradientBoostingRegressor()
gbm.fit(xtrain_final, ytrain)
gbmpred = gbm.predict(xtest_final)
validation(ytest, gbmpred)

In [ ]:
feature_imp = pd.DataFrame()
feature_imp['Feature']=xtrain_final.columns
feature_imp['Importance']=gbm.feature_importances_

feature_imp = feature_imp.sort_values('Importance',ascending=False)
sns.barplot(x='Importance', y='Feature', data=feature_imp)
plt.show()

In [ ]:
adb = AdaBoostRegressor()
adb.fit(xtrain_final, ytrain)
adbpred = adb.predict(xtest_final)
validation(ytest, adbpred)

In [ ]:
xgb = XGBRegressor()
xgb.fit(xtrain_final, ytrain)
xgbpred = xgb.predict(xtest_final)
validation(ytest, xgbpred)

In [ ]:
import pickle

model_rf = (rf)

# Save the tuple to a pickle file
with open('model_rf.pkl', 'wb') as f:
    pickle.dump(model_rf, f)

* **we conclude that our final model is Random Forest Because of simplicity and better performance.**

**Using Random Forest Our performance is:**

* R-Squared : 0.9762034521686348

* Mean Squared Error : 2238210.904146328

* Root Mean Squared Error : 1496.065140341933

# ***Thank you!...***